# Building a Multi-Agent System(MAS) with MCP

Copyright 2025-2026, Denis Rothman

Trong sổ tay này, chúng ta sẽ xây dựng một hệ thống đa tác nhân từ đầu. Chúng ta sẽ sử dụng educational version của Giao thức Ngữ cảnh Mô hình (Model Context Protocol) để các tác nhân của chúng ta giao tiếp với nhau. Bạn sẽ học cách xác định một định dạng tin nhắn chuẩn. Bạn sẽ tạo ra các tác nhân AI chuyên biệt như một nhà nghiên cứu và một nhà văn. Bạn cũng sẽ xây dựng một bộ điều phối (orchestrator) để quản lý quy trình làm việc của chúng. Bằng cách làm theo các bước hướng dẫn, bạn sẽ thấy cách các thành phần này hoạt động cùng nhau. Chúng ta sẽ biến một mục tiêu cấp cao thành một sản phẩm hoàn chỉnh — ví dụ như một bài đăng trên blog. Điều này giúp bạn hiểu trực tiếp các ý tưởng cốt lõi đằng sau các hệ thống trí tuệ nhân tạo hợp tác.

In [1]:

#@title 1.Initializing the Client
# ------------------------------------------------------------------------------
# We'll need the `openai` library to communicate with the LLM.
# Note: This notebook assumes you have already run a setup cell in your Colab
# environment to load your API key from Colab Secrets into an environment
# variable, as you specified.
# ------------------------------------------------------------------------------

import json
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

# --- Initialize the OpenAI Client ---
# The client will automatically read the OPENAI_API_KEY from your environment.
client = OpenAI()
print("OpenAI client initialized.")

OpenAI client initialized.


In [2]:
#@title 2.Defining the Protocol: The MCP Standard
# ------------------------------------------------------------------------------
# Before we build our agents, we must define the language they will speak.
# MCP provides a simple, structured way to pass context. For this example,
# our MCP message will be a Python dictionary with key fields.
# ------------------------------------------------------------------------------
def create_mcp_message(sender, content, metadata=None):
    """Creates a standardized MCP message."""
    return {
        "protocol_version": "1.0",
        "sender": sender,
        "content": content,
        "metadata": metadata or {}
    }

print("--- Example MCP Message (Our Simplified Version) ---")
example_mcp = create_mcp_message(
    sender="Orchestrator",
    content="Research the benefits of the Mediterranean diet.",
    metadata={"task_id": "T-123", "priority": "high"}
)
print(json.dumps(example_mcp, indent=2))

--- Example MCP Message (Our Simplified Version) ---
{
  "protocol_version": "1.0",
  "sender": "Orchestrator",
  "content": "Research the benefits of the Mediterranean diet.",
  "metadata": {
    "task_id": "T-123",
    "priority": "high"
  }
}


In [3]:
#@title 3.Building the Agents: The Specialists
# ------------------------------------------------------------------------------
# Each agent is a function that takes an MCP message as input and returns one
# as output. The core of each agent is a carefully crafted "Semantic Blueprint"
# in the system prompt that defines its persona and task.
# ------------------------------------------------------------------------------

def call_llm(system_prompt, user_content):
    """A helper function to call the OpenAI API using the new client syntax."""
    try:
        # Using the updated client.chat.completions.create method
        response = client.chat.completions.create(
            model="gpt-5.2",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_content}
            ]
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"An error occurred with the API call: {e}"

## Agent 1: The Researcher

In [4]:
def researcher_agent(mcp_input):
    """
    This agent takes a research topic, finds information, and returns a summary.
    """
    print("\n[Researcher Agent Activated]")

    # chế độ ăn Địa Trung Hải (mediterranean diet): một mô hình dinh dưỡng đặc trưng bởi các thực phẩm truyền thống 
    # của các quốc gia ven biển Địa Trung Hải như ô liu, cá, rau củ và ngũ cốc nguyên hạt
    simulated_database = {
        "mediterranean diet": "Chế độ ăn Địa Trung Hải giàu trái cây, rau củ, ngũ cốc nguyên hạt, dầu ô liu và cá. Các nghiên cứu cho thấy chế độ ăn này liên quan đến nguy cơ mắc bệnh tim mạch thấp hơn, cải thiện sức khỏe não bộ và kéo dài tuổi thọ. Các thành phần chủ chốt bao gồm chất béo không bão hòa đơn và chất chống oxy hóa."
    }
    research_topic = mcp_input['content']
    research_result = simulated_database.get(research_topic.lower(), "Không tìm thấy thông tin nào về chủ đề này.")
    system_prompt = """
    Bạn là một Research Analyst chuyên nghiệp. Nhiệm vụ của bạn là tổng hợp (synthesize) các thông tin được cung cấp thành 3-4 bullet points súc tích. Hãy tập trung hoàn toàn vào các key findings quan trọng nhất, đảm bảo nội dung cô đọng và có giá trị phân tích cao.
    """

    summary = call_llm(system_prompt, research_result)
    print(f"Research summary created for: '{research_topic}'")
    return create_mcp_message(
        sender="ResearcherAgent",
        content=summary,
        metadata={"source": "Simulated Internal DB"}
    )

## Agent 2: The Writer

In [5]:
def writer_agent(mcp_input):
    """
    This agent takes research findings and writes a short blog post.
    """
    print("\n[Writer Agent Activated]")
    research_summary = mcp_input['content']
    system_prompt = "Bạn là một người viết nội dung lành nghề cho một blog về sức khỏe và chăm sóc bản thân. Giọng văn của bạn hấp dẫn, giàu thông tin và mang tính khuyến khích. Nhiệm vụ của bạn là lấy các điểm nghiên cứu sau đây và viết một bài blog ngắn, lôi cuốn (khoảng 150 từ) với một tiêu đề bắt mắt."
    blog_post = call_llm(system_prompt, research_summary)
    print("Blog post drafted.")
    return create_mcp_message(
        sender="WriterAgent",
        content=blog_post,
        metadata={"word_count": len(blog_post.split())}
    )

In [6]:
#@title 4.Building the Orchestrator: The Project Manager
# ------------------------------------------------------------------------------
# The Orchestrator manages the workflow. It calls the agents in the correct
# order, passing context from one to the next using MCP messages.
# ------------------------------------------------------------------------------

def orchestrator(initial_goal):
    """
    Manages the multi-agent workflow to achieve a high-level goal.
    """
    print("="*50)
    print(f"[Orchestrator] Goal Received: '{initial_goal}'")
    print("="*50)

    # --- Step 1: Orchestrator plans and calls the Researcher Agent ---
    print("\n[Orchestrator] Task 1: Research. Delegating to Researcher Agent.")
    research_topic = "Mediterranean Diet"
    mcp_to_researcher = create_mcp_message(
        sender="Orchestrator",
        content=research_topic
    )
    mcp_from_researcher = researcher_agent(mcp_to_researcher)
    print("\n[Orchestrator] Research complete. Received summary:")
    print("-" * 20)
    print(mcp_from_researcher['content'])
    print("-" * 20)

    # --- Step 2: Orchestrator calls the Writer Agent ---
    print("\n[Orchestrator] Task 2: Write Content. Delegating to Writer Agent.")
    mcp_to_writer = create_mcp_message(
        sender="Orchestrator",
        content=mcp_from_researcher['content']
    )
    mcp_from_writer = writer_agent(mcp_to_writer)
    print("\n[Orchestrator] Writing complete.")

    # --- Step 3: Orchestrator presents the final result ---
    final_output = mcp_from_writer['content']
    print("\n" + "="*50)
    print("[Orchestrator] Workflow Complete. Final Output:")
    print("="*50)
    print(final_output)

In [7]:
#@title 5.Run the System
# ------------------------------------------------------------------------------
# Let's give our Orchestrator a high-level goal and watch the agent team work.
# ------------------------------------------------------------------------------
# Note: the goal is hard coded for this exmaple before we make it a variable in the subsequent notebooks
user_goal = "Create a blog post about the benefits of the Mediterranean diet."
orchestrator(user_goal)

[Orchestrator] Goal Received: 'Create a blog post about the benefits of the Mediterranean diet.'

[Orchestrator] Task 1: Research. Delegating to Researcher Agent.

[Researcher Agent Activated]
Research summary created for: 'Mediterranean Diet'

[Orchestrator] Research complete. Received summary:
--------------------
- Chế độ ăn Địa Trung Hải nhấn mạnh thực phẩm toàn phần: trái cây, rau củ, ngũ cốc nguyên hạt, dầu ô liu và cá.  
- Bằng chứng nghiên cứu cho thấy mối liên hệ với **giảm nguy cơ bệnh tim mạch**, đồng thời **cải thiện sức khỏe não bộ** và **tăng tuổi thọ**.  
- Lợi ích sức khỏe được cho là đến từ các **thành phần cốt lõi** như **chất béo không bão hòa đơn** (đặc biệt từ dầu ô liu) và **chất chống oxy hóa** (từ rau quả, thực phẩm nguyên hạt).  
- Tác động tổng thể phản ánh sự kết hợp giữa **mẫu hình ăn uống** (dietary pattern) và **chất lượng chất béo/vi chất** hơn là một “thực phẩm đơn lẻ” tạo ra hiệu quả.
--------------------

[Orchestrator] Task 2: Write Content. Delegatin